In [1]:
from interpolate import *
from evaluate import *

In [2]:
conditions = {
    "DATASET_DESCRIPTION": {},
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "PCA_Flat"}
}

base_folder = "Output/Reproduce"
output_folder = "Scripts/"

In [3]:
predictions_list, descriptions_list = load_compatible_available_runs(base_folder, conditions)

1 matching runs found


In [4]:
interpolate_predictions(descriptions_list[0], predictions_list[0], output_folder)

['.ipynb_checkpoints', 'default_grid.txt', 'grid_description_r_5_nbs_5_ico.txt', 'grid_description_r_5_nbs_6_ico.txt', 'ico_to_model.sh', 'model_to_ico.sh', 'run_ico_to_model.ipynb', 'run_model_to_ico.ipynb', 'tmp.nc']
Scripts/ Scripts/model_to_ico.sh


OSError: [WinError 193] %1 ist keine zulässige Win32-Anwendung